In [1]:
%matplotlib inline 
# from gail.train import _train, _trainGAIL, gen_expert_data
from gail.dataloader import *
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



class Struct:
    def __init__(self, **entries):
        self.__dict__.update(entries)
        
args={'episodes':5000, 
      'seed':0, 
      'steps':100, 
      'batch_size':10,
      'epochs':2000,
      'model_directory':'models/',
      'data_directory':'D:/Michael/Learning/duckietown_data/',
      'get_samples':False}
args = Struct(**args)

# print(args.episodes)
# gen_expert_data(args)
def run():
    # torch.multiprocessing.freeze_support()
    # print('loop')
    data = ExpertTrajDataset(args)
    dataloader = DataLoader(data, batch_size=args.batch_size, shuffle=True, num_workers=4)


    for i_batch, sample_batched in enumerate(dataloader):
        sample_batched_obs = sample_batched['observation'].float().to(device).reshape((args.batch_size*100, 3, 160,120))
        sample_batched_acts= sample_batched['action'].float().to(device).reshape((args.batch_size*100, 2))
        for i,e in enumerate(sample_batched_obs):
            plt.figure(i)
#             print(i[0,:,:])
            plt.imshow(e[0,:,:].to('cpu').numpy())
        print(i_batch, sample_batched_obs.size(), sample_batched_acts.size())


        if i_batch == 4:
            break

    # _trainGAIL(args)
    # _train(args)

# run()

In [2]:
from learning.utils.env import launch_env
from learning.utils.wrappers import NormalizeWrapper, ImgWrapper, \
    DtRewardWrapper, ActionWrapper, ResizeWrapper
from learning.utils.teacher import PurePursuitExpert
from gail.models import *

env = launch_env()
env = ResizeWrapper(env)
env = NormalizeWrapper(env) 
env = ImgWrapper(env)
env = ActionWrapper(env)
env = DtRewardWrapper(env)
print("Initialized Wrappers")


observation_shape = (None, ) + env.observation_space.shape
action_shape = (None, ) + env.action_space.shape

model = Generator(action_dim=2)
model.load_state_dict(torch.load('test.pt'))
model.eval()
# model.to(device)
# Create an imperfect demonstrator

observations = []
actions = []

# let's collect our samples
for episode in range(0, args.episodes):
    print("Starting episode", episode)
    observation = env.reset()
    for steps in range(0, args.steps):
        # use our 'expert' to predict the next action.
        observation = torch.tensor(observation, device="cpu").float()
        action = model.forward(observation.unsqueeze(0)).detach().numpy()[0]
        print(action)
        observation, reward, done, info = env.step(action)
        observations.append(observation)
        actions.append(action)
        env.render()
    env.reset()
env.close()


INFO:gym-duckietown:gym-duckietown 2019.0.0

INFO:gym-duckietown:Registering gym environment id: Duckietown-4way-v0
INFO:gym-duckietown:Registering gym environment id: Duckietown-loop_dyn_duckiebots-v0
INFO:gym-duckietown:Registering gym environment id: Duckietown-loop_empty-v0
INFO:gym-duckietown:Registering gym environment id: Duckietown-loop_obstacles-v0
INFO:gym-duckietown:Registering gym environment id: Duckietown-loop_pedestrians-v0
INFO:gym-duckietown:Registering gym environment id: Duckietown-small_loop-v0
INFO:gym-duckietown:Registering gym environment id: Duckietown-small_loop_cw-v0
INFO:gym-duckietown:Registering gym environment id: Duckietown-straight_road-v0
INFO:gym-duckietown:Registering gym environment id: Duckietown-udem1-v0
INFO:gym-duckietown:Registering gym environment id: Duckietown-zigzag_dists-v0
C:\Users\micha\miniconda3\envs\gym-duckietown\lib\site-packages\gym\logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(co

DEBUG:gym-duckietown:[3.50186974 0.         2.10261411] corresponds to tile at (5, 3) which is not drivable: {'coords': (5, 3), 'kind': 'floor', 'angle': 0, 'drivable': False, 'texture': <gym_duckietown.graphics.Texture object at 0x000001A3AE5F2278>, 'color': array([1, 1, 1])}
DEBUG:gym-duckietown:Invalid pose. Collision free: True On drivable area: False
DEBUG:gym-duckietown:safety_factor: 1.3
DEBUG:gym-duckietown:pos: [3.5769519  0.         2.04041348]
DEBUG:gym-duckietown:l_pos: [3.50186974 0.         2.10261411]
DEBUG:gym-duckietown:r_pos: [3.65203407 0.         1.97821284]
DEBUG:gym-duckietown:f_pos: [3.50231114 0.         1.95031488]
DEBUG:gym-duckietown:[3.47260658 0.         1.77460142] corresponds to tile at (5, 3) which is not drivable: {'coords': (5, 3), 'kind': 'floor', 'angle': 0, 'drivable': False, 'texture': <gym_duckietown.graphics.Texture object at 0x000001A3AE5F2278>, 'color': array([1, 1, 1])}
DEBUG:gym-duckietown:Invalid pose. Collision free: True On drivable area: 

Initialized Wrappers


DEBUG:gym-duckietown:[2.33703727 0.         2.57700384] corresponds to tile at (3, 4) which is not drivable: {'coords': (3, 4), 'kind': 'floor', 'angle': 0, 'drivable': False, 'texture': <gym_duckietown.graphics.Texture object at 0x000001A3AE5F2278>, 'color': array([1, 1, 1])}
DEBUG:gym-duckietown:Invalid pose. Collision free: True On drivable area: False
DEBUG:gym-duckietown:safety_factor: 1.3
DEBUG:gym-duckietown:pos: [2.33703727 0.         2.57700384]
DEBUG:gym-duckietown:l_pos: [2.24568806 0.         2.542922  ]
DEBUG:gym-duckietown:r_pos: [2.42838647 0.         2.61108568]
DEBUG:gym-duckietown:f_pos: [2.37793548 0.         2.46738479]
DEBUG:gym-duckietown:[2.28873639 0.         2.68406941] corresponds to tile at (3, 4) which is not drivable: {'coords': (3, 4), 'kind': 'floor', 'angle': 0, 'drivable': False, 'texture': <gym_duckietown.graphics.Texture object at 0x000001A3AE5F2278>, 'color': array([1, 1, 1])}
DEBUG:gym-duckietown:Invalid pose. Collision free: True On drivable area: 

Starting episode 0


...

INFO:gym-duckietown:Pos: [2.63183152 0.         3.0006965 ] angle 11.335911394625716
INFO:gym-duckietown:Pos: [2.63183152 0.         3.0006965 ] angle 11.335911394625716


[0.79166526 1.4184787 ]


INFO:gym-duckietown:Pos: [2.64515884 0.         3.03395356] angle 11.417519249849079
INFO:gym-duckietown:Pos: [2.64515884 0.         3.03395356] angle 11.417519249849079


[0.7919    1.3723402]


KeyboardInterrupt: 

In [4]:
import torch
actions = torch.load('data_a.pt')
observations = torch.load('data_o.pt')

actions.max()

5.702396438546007

In [14]:
import numpy as np
import torch

j = np.ones(3)
k+=j
k

TypeError: Required argument 'object' (pos 1) not found

In [12]:
torch.stack((k,j))
k

TypeError: expected Tensor as element 0 in argument 0, but got list